In [3]:
import json
# read in JSON file with restricted access data
# restricted API key is located in a file containing JSON encoded text in which
# the API key is the value of a dictionary with a key of API_KEY
apik = ""
with open('/home/edith/gitpracpy2/dontpushme/security.json', 'r') as sensitivefile:
    s = json.loads(sensitivefile.read())
    apik = s['API_KEY']
print(apik)


38473847387affccef4636


In [4]:
import requests


# Connect to the NHTSA API by building a URL-encoded GET request
# API Endpoint: "front door our API server"
API_ENDPOINT = "http://www.nhtsa.gov/webapi/api/"

# Sample request URL for testing
# http://www.nhtsa.gov/webapi/api/Recalls/vehicle/modelyear/2000/make/saturn/model/ls?format=json

# use requests to ask a remote server for data and store the response in var response
headers = {'user-agent': 'loretta_on_python'}
response = requests.get('http://www.nhtsa.gov/webapi/api/Recalls/vehicle/modelyear/2000/make/saturn/model/ls?format=json', headers=headers)
# when we get our response back, we'll check it status to see if all is well 
# before unpacking the data
print(response.status_code) 
# 200 is OK, 
# 404 means "I don't have that file", 
# 500-level codes are server errors
# print(response.text)
payload_objects = json.loads(response.text)
# Extract just the value of the key Results, which is a list
recalls = payload_objects['Results']
# iterate over list
for i in recalls:
    print('************* RECALL *****************')
    print(i)

200
************* RECALL *****************
{'Manufacturer': 'GENERAL MOTORS CORP.', 'NHTSACampaignNumber': '00V228003', 'ReportReceivedDate': '/Date(965620800000-0400)/', 'Component': 'SEAT BELTS', 'Summary': 'VEHICLE DESCRIPTION:  PASSENGER VEHICLES, LIGHT DUTY TRUCKS, SPORT UTILITY VEHICLES, AND MINI VANS EQUIPPED WITH TRW SEAT BELT BUCKLE ASSEMBLIES.  THE BUCKLE BASE OF THESE SEAT BELT ASSEMBLIES WERE NOT PROPERLY HEAT TREATED AND THEREFORE DO NOT PASS THE LOAD BEARING REQUIREMENT OF FEDERAL MOTOR VEHICLE SAFETY STANDARD NO. 209, "SEAT BELT ASSEMBLIES."', 'Conequence': 'IN THE EVENT OF A VEHICLE CRASH, THE OCCUPANT MAY NOT BE PROPERLY RESTRAINED.', 'Remedy': "DEALERS WILL INSPECT THE VEHICLE'S SAFETY BELT BUCKLE ASSEMBLY DATE CODES AND, IF NECESSARY, WILL REPLACE ANY SAFETY BELT BUCKLE ASSEMBLY.", 'Notes': "OWNER NOTIFICATION BEGAN AUGUST 24, 2000.   OWNERS WHO TAKE THEIR VEHICLES TO AN AUTHORIZED DEALER ON AN AGREED UPON SERVICE DATE AND DO NOT RECEIVE THE FREE REMEDY WITHIN A REAS

In [9]:
def build_nhtsa_url(year, make, model):
    '''
    Composes an API request URL to the NHTSA given a vehicle's year, make, and model
    '''
    API_ENDPOINT = "http://www.nhtsa.gov/webapiz/api/"
    recall_portion = ""
    # Use Python string formatting to create injecetion points with %s
    # python will be expecting a tuple following this string which contains a 
    # string value for each of the %s found in base string
    if model is not None:
        recall_portion = "Recalls/vehicle/modelyear/%s/make/%s/model/%s?format=json" % (year, make, model)
    else:
        recall_portion = "Recalls/vehicle/modelyear/%s/make/%s/?format=json" % (year, make)
    full_url = API_ENDPOINT + recall_portion
    print("URL: ", full_url)
    return full_url
    
    

In [10]:
def make_api_req(url):
    # send the request over the network and store response in resp
    resp = requests.get(url)
    print("Made request, response status: ", resp.status_code)
    # check for status code 200
    if(int(resp.status_code) == 200):
        payload_objects = json.loads(resp.text)
        return payload_objects
    else:
        raise Exception("Non 200 status code")

In [12]:
# test our URL builder:
try:
    print(make_api_req(build_nhtsa_url(2005, 'Ford', "MUSTANG")))
except Exception as ex:
    print(ex)

URL:  http://www.nhtsa.gov/webapiz/api/Recalls/vehicle/modelyear/2005/make/Ford/model/MUSTANG?format=json
Made request, response status:  404
Non 200 status code
